In [37]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt # Vẫn cần cho LIME plot nếu muốn hiển thị trong môi trường hỗ trợ
import pandas as pd
from lime.lime_tabular import LimeTabularExplainer
import re # Regular expressions cho NLP đơn giản
import logging
import warnings


In [39]:
symptom_names_expanded = [
    "Fever", "Cough", "Sore Throat", "Headache", "Fatigue", "Runny Nose",
    "Shortness of Breath", "Chest Pain", "Chills", "Nausea"
]
num_symptoms_expanded = len(symptom_names_expanded)

diseases_expanded = ["Flu", "Cold", "COVID-19", "Allergy", "Bronchitis", "Pneumonia"]
num_diseases_expanded = len(diseases_expanded)

# Dữ liệu huấn luyện (giống V4.1)
X_train_expanded = np.array([
    [3, 2, 1, 2, 3, 1, 0, 0, 2, 1], [1, 2, 2, 1, 1, 3, 0, 0, 0, 0],
    [3, 3, 2, 2, 3, 1, 2, 1, 2, 1], [0, 1, 1, 1, 1, 2, 1, 0, 0, 0],
    [1, 3, 1, 0, 2, 1, 1, 2, 1, 0], [3, 3, 0, 1, 3, 0, 3, 2, 3, 1]
], dtype=np.float32)
y_train_expanded_indices = np.array([0, 1, 2, 3, 4, 5], dtype=np.int32)

In [41]:
# Data Augmentation (giống V4.1)
def augment_data_expanded_v5(X, y, n_augmentations_per_sample=10, max_change=1):
    X_augmented = list(X)
    y_augmented = list(y)
    for _ in range(n_augmentations_per_sample):
        for i in range(len(X)):
            sample = X[i].copy()
            num_symptoms_to_change = np.random.randint(1, 4)
            for _ in range(num_symptoms_to_change):
                idx_to_change = np.random.randint(0, X.shape[1])
                change = np.random.randint(-max_change, max_change + 1)
                new_value = sample[idx_to_change] + change
                sample[idx_to_change] = np.clip(new_value, 0, 3)
            X_augmented.append(sample)
            y_augmented.append(y[i])
    return np.array(X_augmented, dtype=np.float32), np.array(y_augmented, dtype=np.int32)

X_train_aug_exp, y_train_indices_aug_exp = augment_data_expanded_v5(X_train_expanded, y_train_expanded_indices, n_augmentations_per_sample=20)
y_train_aug_exp = tf.keras.utils.to_categorical(y_train_indices_aug_exp, num_classes=num_diseases_expanded)

In [43]:
def build_model_expanded_for_lime_v5():
    inputs = tf.keras.Input(shape=(num_symptoms_expanded,))
    x = tf.keras.layers.Dense(64, activation='relu')(inputs)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(num_diseases_expanded, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

chatbot_model = build_model_expanded_for_lime_v5()
print("--- Training Chatbot Model ---")
chatbot_model.fit(X_train_aug_exp, y_train_aug_exp, epochs=150, verbose=0)
print("Chatbot Model trained.")

# LIME (giống V4.1)
feature_names_exp = symptom_names_expanded
class_names_exp = diseases_expanded


--- Training Chatbot Model ---
Chatbot Model trained.


In [44]:
def predict_fn_chatbot_lime(numpy_array_of_symptoms):
    return chatbot_model.predict(numpy_array_of_symptoms, verbose=0)

explainer_chatbot_lime = LimeTabularExplainer(
    training_data=X_train_aug_exp,
    feature_names=feature_names_exp,
    class_names=class_names_exp,
    mode='classification',
    discretizer='quartile',
    verbose=False
)


In [45]:
# ---- NLP Parser Đơn giản và Logic Chatbot ----

# Từ điển ánh xạ từ đồng nghĩa hoặc cách nói khác về triệu chứng sang tên chuẩn
symptom_synonyms = {
    # Fever
    "sốt": "Fever", "nóng": "Fever", "thân nhiệt cao": "Fever", "người nóng": "Fever",
    "cảm thấy nóng": "Fever", "bị sốt": "Fever", "lên cơn sốt": "Fever",

    # Cough
    "ho": "Cough", "bị ho": "Cough", "ho khan": "Cough", "ho có đờm": "Cough", # (Model hiện tại chưa phân biệt loại ho)
    "ho nhiều": "Cough", "ho ít": "Cough",

    # Sore Throat
    "đau họng": "Sore Throat", "rát họng": "Sore Throat", "viêm họng": "Sore Throat",
    "cổ họng đau": "Sore Throat", "ngứa họng": "Sore Throat", "khô họng": "Sore Throat",

    # Headache
    "đau đầu": "Headache", "nhức đầu": "Headache", "đau nửa đầu": "Headache",
    "đầu đau": "Headache", "nặng đầu": "Headache",

    # Fatigue
    "mệt mỏi": "Fatigue", "uể oải": "Fatigue", "kiệt sức": "Fatigue", "không có sức": "Fatigue",
    "đuối sức": "Fatigue", "cảm thấy mệt": "Fatigue", "người mệt": "Fatigue", "chán nản": "Fatigue",

    # Runny Nose
    "sổ mũi": "Runny Nose", "chảy nước mũi": "Runny Nose", "nghẹt mũi": "Runny Nose", # (Nghẹt mũi có thể là triệu chứng riêng)
    "mũi chảy nước": "Runny Nose", "khụt khịt": "Runny Nose",

    # Shortness of Breath
    "khó thở": "Shortness of Breath", "thở gấp": "Shortness of Breath", "hụt hơi": "Shortness of Breath",
    "thở không ra hơi": "Shortness of Breath", "thở nông": "Shortness of Breath", "ngạt thở": "Shortness of Breath",

    # Chest Pain
    "đau ngực": "Chest Pain", "tức ngực": "Chest Pain", "nặng ngực": "Chest Pain",
    "đau ở ngực": "Chest Pain", "khó chịu ở ngực": "Chest Pain",

    # Chills
    "ớn lạnh": "Chills", "rét run": "Chills", "lạnh run": "Chills", "cảm thấy lạnh": "Chills",
    "nổi da gà": "Chills",

    # Nausea
    "buồn nôn": "Nausea", "ói": "Nausea", "muốn ói": "Nausea", "nôn nao": "Nausea",
    "lợm giọng": "Nausea", "khó chịu ở bụng": "Nausea", "cảm giác muốn nôn": "Nausea",
}
# Từ điển ánh xạ từ mô tả mức độ sang giá trị số
level_map = {
    "không": 0, "không có": 0, "ko": 0,
    "nhẹ": 1, "ít": 1, "hơi": 1, "một chút": 1,
    "vừa": 2, "trung bình": 2, "khá": 2,
    "nặng": 3, "nhiều": 3, "rất": 3, "cao": 3 # "sốt cao"
}

In [46]:
def parse_symptoms_from_text(text, current_symptoms_vector):
    """
    Phân tích văn bản đầu vào để trích xuất triệu chứng và mức độ.
    Cập nhật current_symptoms_vector.
    Trả về True nếu có ít nhất một triệu chứng được nhận diện, False nếu không.
    """
    text = text.lower()
    symptoms_found_in_input = False

    # Ưu tiên các cụm (mức độ + triệu chứng) hoặc (triệu chứng + mức độ)
    for level_word, level_val in level_map.items():
        for syn_word, symptom_name in symptom_synonyms.items():
            # Pattern: level_word + syn_word (e.g., "sốt cao", "ho nhẹ")
            pattern1 = rf"\b{level_word}\s+{syn_word}\b"
            # Pattern: syn_word + level_word (e.g., "sốt cao", "ho nhẹ")
            pattern2 = rf"\b{syn_word}\s+{level_word}\b"
            
            if re.search(pattern1, text) or re.search(pattern2, text):
                if symptom_name in symptom_names_expanded:
                    idx = symptom_names_expanded.index(symptom_name)
                    current_symptoms_vector[idx] = float(level_val)
                    symptoms_found_in_input = True
                    # Loại bỏ cụm đã xử lý để tránh xử lý lại
                    text = re.sub(pattern1, "", text)
                    text = re.sub(pattern2, "", text)

    # Xử lý các triệu chứng đứng một mình (mặc định mức độ là 2 - "vừa" hoặc "có")
    for syn_word, symptom_name in symptom_synonyms.items():
        pattern = rf"\b{syn_word}\b"
        if re.search(pattern, text):
            if symptom_name in symptom_names_expanded:
                idx = symptom_names_expanded.index(symptom_name)
                # Nếu triệu chứng này chưa được gán mức độ từ cụm (level + symptom), gán mặc định
                if current_symptoms_vector[idx] == 0: # Chỉ gán nếu chưa có
                     current_symptoms_vector[idx] = 2.0 # Mặc định là "vừa"
                symptoms_found_in_input = True

    return symptoms_found_in_input


In [47]:
def get_missing_symptoms_questions(symptoms_vector):
    """Tạo câu hỏi cho các triệu chứng chưa được cung cấp (giá trị là 0)."""
    questions = []
    for i, symptom_name in enumerate(symptom_names_expanded):
        if symptoms_vector[i] == 0: # Chỉ hỏi nếu chưa có thông tin
             # Đơn giản hóa câu hỏi
            if symptom_name == "Fever": questions.append(f"Bạn có bị sốt không (không/nhẹ/vừa/nặng)?")
            elif symptom_name == "Cough": questions.append(f"Bạn có ho không (không/nhẹ/vừa/nặng)?")
            elif symptom_name == "Sore Throat": questions.append(f"Bạn có đau họng không (không/nhẹ/vừa/nặng)?")
            # ... thêm các câu hỏi thân thiện hơn cho các triệu chứng khác
            else:
                questions.append(f"Bạn có bị {symptom_name.lower()} không (không/nhẹ/vừa/nặng)?")
    # Chỉ hỏi một vài triệu chứng mỗi lần để không làm người dùng quá tải
    return questions[:3] # Hỏi tối đa 3 triệu chứng còn thiếu


In [48]:
def chatbot_interaction():
    print("Xin chào! Tôi là chatbot chẩn đoán sức khỏe. Hãy mô tả các triệu chứng của bạn.")
    print("Ví dụ: 'tôi bị sốt cao và ho nhiều', hoặc 'đau họng nhẹ'.")
    print("Bạn có thể nói 'xong' hoặc 'chẩn đoán' khi đã cung cấp đủ triệu chứng.")
    print("Gõ 'thoát' để kết thúc.")

    current_symptoms = np.zeros(num_symptoms_expanded, dtype=np.float32)
    asked_symptoms_in_session = set() # Theo dõi các triệu chứng đã hỏi trong phiên

    while True:
        user_input = input("Bạn: ").strip()

        if not user_input:
            continue
        if user_input.lower() in ['thoát', 'exit', 'quit']:
            print("Chatbot: Cảm ơn bạn đã sử dụng dịch vụ. Tạm biệt!")
            break

        symptoms_identified = parse_symptoms_from_text(user_input, current_symptoms)

        if user_input.lower() in ['xong', 'chẩn đoán', 'chan doan', 'ok', 'xem kết quả'] or \
           (symptoms_identified and np.any(current_symptoms > 0) and len(get_missing_symptoms_questions(current_symptoms)) == 0) or \
           (not symptoms_identified and np.any(current_symptoms > 0)): # Nếu người dùng không cung cấp thêm triệu chứng mới nhưng đã có sẵn

            if not np.any(current_symptoms > 0):
                print("Chatbot: Tôi chưa nhận được thông tin triệu chứng nào. Bạn có thể mô tả lại được không?")
                continue

            print("\nChatbot: Dựa trên các triệu chứng bạn cung cấp:")
            for i, val in enumerate(current_symptoms):
                if val > 0:
                    print(f"  - {symptom_names_expanded[i]}: mức độ {int(val)}")
            
            print("\nChatbot: Đang phân tích...")
            # Dự đoán
            patient_vector = current_symptoms.reshape(1, -1)
            probabilities = predict_fn_chatbot_lime(patient_vector)[0]
            
            print("\n--- Kết quả chẩn đoán ---")
            sorted_indices = np.argsort(probabilities)[::-1]
            for i in range(min(3, num_diseases_expanded)): # Hiển thị top 3
                idx = sorted_indices[i]
                print(f"  - {class_names_exp[idx]}: {probabilities[idx]*100:.2f}%")
            
            primary_diagnosis_idx = sorted_indices[0]
            primary_diagnosis_name = class_names_exp[primary_diagnosis_idx]
            print(f"\nChẩn đoán chính có khả năng cao nhất: {primary_diagnosis_name}")

            # Giải thích LIME
            print(f"\n--- Giải thích cho chẩn đoán {primary_diagnosis_name} ---")
            explanation = explainer_chatbot_lime.explain_instance(
                data_row=current_symptoms,
                predict_fn=predict_fn_chatbot_lime,
                num_features=7, # Số feature quan trọng
                top_labels=1 # Chỉ giải thích cho chẩn đoán chính
            )
            # Hiển thị giải thích LIME dạng text
            print("Các yếu tố ảnh hưởng đến chẩn đoán này (theo LIME):")
            for feature_desc, weight in explanation.as_list(label=primary_diagnosis_idx):
                # feature_desc có thể là "Symptom <= Value" hoặc "Value < Symptom <= Value2" hoặc "Symptom > Value"
                # Chúng ta cần làm cho nó dễ đọc hơn
                readable_feature = feature_desc.replace(" <= ", " ở mức tối đa là ")
                readable_feature = readable_feature.replace(" < ", " lớn hơn ")
                readable_feature = readable_feature.replace("Sore Throat", "Đau họng") # Ví dụ
                # ... thêm các thay thế khác cho dễ đọc
                
                if weight > 0:
                    print(f"  - '{readable_feature}' làm TĂNG khả năng bị {primary_diagnosis_name} (ảnh hưởng: {weight:.2f})")
                else:
                    print(f"  - '{readable_feature}' làm GIẢM khả năng bị {primary_diagnosis_name} (ảnh hưởng: {weight:.2f})")
            
            # Reset cho phiên mới hoặc hỏi tiếp
            print("\nChatbot: Bạn có muốn chẩn đoán cho một bộ triệu chứng khác không? (gõ 'có' hoặc mô tả triệu chứng mới, 'không' để thoát)")
            next_action = input("Bạn: ").strip().lower()
            if next_action == 'có' or next_action == 'co':
                current_symptoms = np.zeros(num_symptoms_expanded, dtype=np.float32)
                asked_symptoms_in_session.clear()
                print("\nChatbot: Hãy mô tả các triệu chứng mới.")
            elif next_action == 'không' or next_action == 'khong':
                print("Chatbot: Cảm ơn bạn đã sử dụng dịch vụ. Tạm biệt!")
                break
            else: # Người dùng có thể nhập triệu chứng mới luôn
                current_symptoms = np.zeros(num_symptoms_expanded, dtype=np.float32)
                asked_symptoms_in_session.clear()
                parse_symptoms_from_text(next_action, current_symptoms)
                # Tiếp tục vòng lặp để hỏi thêm nếu cần hoặc chẩn đoán
        
        elif symptoms_identified:
            print("Chatbot: Tôi đã ghi nhận các triệu chứng bạn vừa cung cấp.")
            # Hiện tại các triệu chứng đã ghi nhận
            print("  Triệu chứng hiện tại:")
            has_symptom = False
            for i, val in enumerate(current_symptoms):
                if val > 0:
                    print(f"    - {symptom_names_expanded[i]}: mức độ {int(val)}")
                    has_symptom = True
            if not has_symptom:
                print("    (Chưa có triệu chứng nào được ghi nhận rõ ràng)")

            missing_questions = get_missing_symptoms_questions(current_symptoms)
            # if missing_questions:
            #     print("Chatbot: Để có chẩn đoán chính xác hơn, bạn có thể cho tôi biết thêm về:")
                # for q_symptom_name in missing_questions:
                #      # Chỉ hỏi nếu triệu chứng đó chưa từng được hỏi trong phiên này
                #     actual_symptom_name = q_symptom_name.split("Bạn có bị ")[1].split(" không")[0].strip() # Trích xuất tên triệu chứng
                #     if actual_symptom_name not in asked_symptoms_in_session:
                #         print(f"  - {q_symptom_name}")
                #         asked_symptoms_in_session.add(actual_symptom_name)
                        
            print("Hoặc bạn có thể nói 'xong' để chẩn đoán với thông tin hiện tại.")
            if missing_questions:
                print("Chatbot: Để có chẩn đoán chính xác hơn, bạn có thể cho tôi biết thêm về:")
                for q_symptom_name in missing_questions:
                    if "Bạn có bị " in q_symptom_name and " không" in q_symptom_name:
                        actual_symptom_name = q_symptom_name.split("Bạn có bị ")[1].split(" không")[0].strip()  # Trích xuất tên triệu chứng
                        if actual_symptom_name not in asked_symptoms_in_session:
                            print(f"  - {q_symptom_name}")
                            asked_symptoms_in_session.add(actual_symptom_name)
                    else:
                        print(f"Chuỗi không hợp lệ: {q_symptom_name}")
                print("Hoặc bạn có thể nói 'xong' để chẩn đoán với thông tin hiện tại.")


            else:
                print("Chatbot: Bạn đã cung cấp đủ thông tin cơ bản. Gõ 'xong' để xem chẩn đoán.")

        else: # Không nhận diện được triệu chứng nào từ input
            if np.any(current_symptoms > 0): # Nếu đã có triệu chứng từ trước
                print("Chatbot: Tôi không nhận diện được thêm triệu chứng nào từ câu nói vừa rồi.")
                print("           Bạn có thể nói 'xong' để chẩn đoán, hoặc cung cấp thêm thông tin.")
            else: # Chưa có triệu chứng nào cả
                print("Chatbot: Xin lỗi, tôi chưa hiểu rõ. Bạn có thể mô tả lại triệu chứng bằng các từ như 'sốt', 'ho', 'đau đầu', kèm theo mức độ (nhẹ, vừa, nặng) được không?")



In [ ]:
# Chạy chatbot
if __name__ == "__main__":
    chatbot_interaction()

Xin chào! Tôi là chatbot chẩn đoán sức khỏe. Hãy mô tả các triệu chứng của bạn.
Ví dụ: 'tôi bị sốt cao và ho nhiều', hoặc 'đau họng nhẹ'.
Bạn có thể nói 'xong' hoặc 'chẩn đoán' khi đã cung cấp đủ triệu chứng.
Gõ 'thoát' để kết thúc.


Bạn:  Tôi bị đau đầu chóng mặt


Chatbot: Tôi đã ghi nhận các triệu chứng bạn vừa cung cấp.
  Triệu chứng hiện tại:
    - Headache: mức độ 2
Hoặc bạn có thể nói 'xong' để chẩn đoán với thông tin hiện tại.
Chatbot: Để có chẩn đoán chính xác hơn, bạn có thể cho tôi biết thêm về:
  - Bạn có bị sốt không (không/nhẹ/vừa/nặng)?
Chuỗi không hợp lệ: Bạn có ho không (không/nhẹ/vừa/nặng)?
Chuỗi không hợp lệ: Bạn có đau họng không (không/nhẹ/vừa/nặng)?
Hoặc bạn có thể nói 'xong' để chẩn đoán với thông tin hiện tại.


Bạn:  tôi bị đau đít



Chatbot: Dựa trên các triệu chứng bạn cung cấp:
  - Headache: mức độ 2

Chatbot: Đang phân tích...

--- Kết quả chẩn đoán ---
  - Allergy: 86.00%
  - Cold: 5.71%
  - COVID-19: 4.00%

Chẩn đoán chính có khả năng cao nhất: Allergy

--- Giải thích cho chẩn đoán Allergy ---
Các yếu tố ảnh hưởng đến chẩn đoán này (theo LIME):
  - 'Fever ở mức tối đa là 1.00' làm TĂNG khả năng bị Allergy (ảnh hưởng: 0.36)
  - 'Runny Nose ở mức tối đa là 1.00' làm GIẢM khả năng bị Allergy (ảnh hưởng: -0.24)
  - 'Shortness of Breath ở mức tối đa là 0.00' làm GIẢM khả năng bị Allergy (ảnh hưởng: -0.22)
  - 'Chills ở mức tối đa là 0.00' làm TĂNG khả năng bị Allergy (ảnh hưởng: 0.15)
  - 'Cough ở mức tối đa là 2.00' làm TĂNG khả năng bị Allergy (ảnh hưởng: 0.10)
  - 'Chest Pain ở mức tối đa là 0.00' làm TĂNG khả năng bị Allergy (ảnh hưởng: 0.08)
  - '1.00 lớn hơn Headache ở mức tối đa là 2.00' làm TĂNG khả năng bị Allergy (ảnh hưởng: 0.05)

Chatbot: Bạn có muốn chẩn đoán cho một bộ triệu chứng khác không? (gõ 'c

Bạn:  có



Chatbot: Hãy mô tả các triệu chứng mới.
